In [1]:
import folium
import geopandas as gpd
import leafmap.foliumap as leafmap
import matplotlib as mpl
import numpy as np
import pandas as pd
import requests


from folium.plugins import GroupedLayerControl



In [2]:
# Nice
url_communes = 'https://www.data.gouv.fr/api/1/datasets/r/7a861138-93c9-44ba-9c9a-2805e9b39b70'
url_pav = 'http://opendata.nicecotedazur.org/data/storage/f/2020-09-02T13%3A44%3A50.928Z/pav.geojson'
url_adresses = 'https://object.files.data.gouv.fr/hydra-geojson/hydra-geojson/d3b9f87e-0f5e-45a8-ba68-efe65d61dc28.geojson'

# Récupération des limites des communes de la métropole

In [3]:
try:
    communes_nice = gpd.read_file('../data/raw/gdf_communes_Nice.shp')
except:
    file = requests.get(url_communes)
    data = file.content
    communes_nice = gpd.read_file(data)
    communes_nice.to_file('../data/raw/gdf_communes_Nice.shp', index = False)

In [4]:
communes_nice.head()

,NOM,CODE_INSEE,geometry
0,LE BROC,06025,"POLYGON ((7.15198 43.84555, 7.15223 43.84554, ..."
1,BONSON,06021,"POLYGON ((7.19215 43.87625, 7.19211 43.87614, ..."
2,GILETTE,06066,"POLYGON ((7.13509 43.86372, 7.13519 43.86362, ..."
3,GATTIERES,06064,"POLYGON ((7.13402 43.79003, 7.1343 43.78975, 7..."
4,ASPREMONT,06006,"POLYGON ((7.24337 43.80202, 7.24351 43.80193, ..."


# Péparation du gdf de pav

In [5]:
# Récupération des data sur les pav
try:
    pav_nice = gpd.read_file('../data/raw/gdf_pav_Nice.shp')
except:
    file = requests.get(url_pav)
    data = file.content
    pav_nice = gpd.read_file(data)
    pav_nice.to_file('../data/raw/gdf_pav_Nice.shp', index = False)

In [6]:
pav_nice.head()

,COMMUNE,JOUR,ADRESSE,CONTENEUR,ID_NCA,TYPE,geometry
0,Nice,Lundi,RUE SORGENTINO / RUE MARECHAL VAUBAN,VERRE,48088V,AERIEN,POINT (7.29189 43.71094)
1,Nice,Lundi,CHEMIN DU MONT GROS / BD BISCHOFFSHEIM (DANS C...,VERRE,49088V,AERIEN,POINT (7.29798 43.7174)
2,Nice,Lundi,BOULEVARD ARMEE DES ALPES FACE AU N°22,VERRE,50088V,AERIEN,POINT (7.29272 43.70805)
3,Nice,Lundi,BD ARMEE DES ALPES / RUE LAURENT GIAUME,VERRE,51088V,AERIEN,POINT (7.29227 43.70668)
4,Nice,Lundi,BOULEVARD PIERRE SOLA ANGLE RUE BEAUTRUCH,VERRE,52088V,AERIEN,POINT (7.28954 43.70601)


In [7]:
# Modification du nom de variable pour uniformisation avec les données d'autres villes
pav_nice = pav_nice.rename({'CONTENEUR': 'Type de flux'}, axis = 1)

In [8]:
pav_nice['Type de flux'].unique()

array(['VERRE', 'PAPIER', 'EMBALLAGES MENAGERS', 'ORDURES MENAGERES',
       'TEXTILE'], dtype=object)

In [9]:
# abbréviation des types de flux pour faciliter la sauvegarde au format .shp

types_dict = {
    'VERRE' : 'V',
    'ORDURES MENAGERES' : 'OM',
    'TEXTILE' : 'T',
    'PAPIER' : 'P',
    'EMBALLAGES MENAGERS' : 'E',
    }

pav_nice['Type de flux'] = pav_nice['Type de flux'].replace(types_dict)

# Préparation du gdf d'adresses

In [10]:
# Récupération des data sur les pav
try:
    adresses_nice = gpd.read_file('../data/raw/gdf_adresses_Nice.shp')
except:
    file = requests.get(url_adresses)
    data = file.content
    adresses_nice = gpd.read_file(data)
    adresses_nice.to_file('../data/raw/gdf_adresses_Nice.shp', index = False)

In [11]:
adresses_nice.head()

,id_ban_com,id_ban_top,id_ban_adr,cle_intero,commune_in,commune_no,commune_de,commune__1,voie_nom,lieudit_co,numero,suffixe,position,x,y,cad_parcel,source,date_der_m,certificat,geometry
0,445d26e1-57df-44db-befe-7351d2926978,be64abc7-42d9-4807-9768-76de15b79cca,0e413c86-6eaa-4174-91bd-7dedb270633e,06013_0115_01500,06013,Belvédère,None,None,CD 71,None,1500,None,entrée,1046322.99,6333137.78,0600130000C0928,commune de Belvédère,2021/06/15 00:00:00,True,POINT (7.32097 44.01194)
1,445d26e1-57df-44db-befe-7351d2926978,4bdfb873-8ad4-4039-a157-93a57bbea3dc,7beb0cd7-3f24-407a-8485-9041be7e4a9f,06013_b101_01141,06013,Belvédère,None,None,Chemin Saint-Grat,None,1141,None,entrée,1051540.66,6338480.01,0600130000H0224|0600130000H0225,commune de Belvédère,2019/11/26 00:00:00,True,POINT (7.38964 44.05736)
2,445d26e1-57df-44db-befe-7351d2926978,4bdfb873-8ad4-4039-a157-93a57bbea3dc,0d2fc8dd-1dfb-4650-8a2b-367309e47a2f,06013_b101_00693,06013,Belvédère,None,None,Chemin Saint-Grat,None,693,None,entrée,1051417.75,6338234.60,0600130000H0235,commune de Belvédère,2019/11/26 00:00:00,True,POINT (7.38794 44.05521)
3,445d26e1-57df-44db-befe-7351d2926978,4bdfb873-8ad4-4039-a157-93a57bbea3dc,5e2e9d38-8a6f-4c59-89bf-2f58d95209ff,06013_b101_00961,06013,Belvédère,None,None,Chemin Saint-Grat,None,961,None,entrée,1051423.94,6338350.46,0600130000H0238,commune de Belvédère,2019/11/26 00:00:00,True,POINT (7.38809 44.05625)
4,445d26e1-57df-44db-befe-7351d2926978,4bdfb873-8ad4-4039-a157-93a57bbea3dc,52846691-2d6a-494a-b6e1-8341ad4e66a1,06013_b101_00965,06013,Belvédère,None,None,Chemin Saint-Grat,None,965,None,entrée,1051425.78,6338354.34,0600130000H0221|0600130000H0222,commune de Belvédère,2019/11/26 00:00:00,True,POINT (7.38812 44.05629)


In [12]:
# extraction des variables d'intérêt
adresses_nice = adresses_nice[['commune_in', 'commune_no', 'geometry']]

# Calcul des distances moyennes adresses/pav

In [13]:
#fonction de calcul de la distance du pav le plus proche d'une adresse
def distances_pav(adresse, gdf_pav, type_pav):
    distance = 10000
    for pav in gdf_pav.loc[gdf_pav['Type de flux'] == type_pav]['geometry']:
        if adresse.distance(pav) < distance:
            distance = adresse.distance(pav)
    return round(distance)



# fonction de calcul de l'ensemble des distances 
def calc_and_save_distances(gdf_adresses, gdf_pav, types_dict, output_file):

    try :
        adresses_with_dist = gpd.read_file(output_file)

    except:
        
        print('Mean distances file not found, generating new file')
        crs_adresses = gdf_adresses.crs
        crs_pav = gdf_pav.crs
        
        adresses_with_dist = gdf_adresses.to_crs(2154)
        gdf_pav = gdf_pav.to_crs(2154)
        
        for type in types_dict.values():
            # ajout, dans le gdf d'adresses, des distances du pav le plus proche pour chaque type de flux
            adresses_with_dist['dist_' + type] = adresses_with_dist['geometry'].apply(lambda x : distances_pav(x, gdf_pav, type))

        adresses_with_dist = adresses_with_dist.to_crs(crs_adresses)
        gdf_pav = gdf_pav.to_crs(crs_pav)
        
        # save
        adresses_with_dist.to_file(output_file, index = False)
    
    return(adresses_with_dist)


adresses_nice = calc_and_save_distances(adresses_nice, pav_nice, types_dict, '../data/cache/distances_Nice')


In [14]:
# groupby par quartier avec distance moyenne
distances_nice = adresses_nice.dissolve(by = 'commune_in', aggfunc = 'mean', numeric_only = True)
        
# jointure avec les données des quartiers
distances_nice = communes_nice.merge(distances_nice.drop(['geometry'], axis = 1), left_on = 'CODE_INSEE', right_on = 'commune_in', how = 'left')

# arrondi des distances à l'entier
for type in types_dict.values():
    dist_type = 'dist_' + str(type)
    distances_nice[dist_type] = distances_nice[dist_type].apply(lambda x : round(x) if pd.notnull(x) else np.nan)


In [15]:
file_to_save = distances_nice.drop(['geometry'], axis = 1)
file_to_save.to_csv('../data/analyse/dist_moy_Nice.csv', index = False)

In [16]:
distances_nice.head()

,NOM,CODE_INSEE,geometry,dist_V,dist_OM,dist_T,dist_P,dist_E
0,LE BROC,06025,"POLYGON ((7.15198 43.84555, 7.15223 43.84554, ...",319,2020,818,352,713
1,BONSON,06021,"POLYGON ((7.19215 43.87625, 7.19211 43.87614, ...",276,3115,2897,279,356
2,GILETTE,06066,"POLYGON ((7.13509 43.86372, 7.13519 43.86362, ...",246,2718,1455,246,280
3,GATTIERES,06064,"POLYGON ((7.13402 43.79003, 7.1343 43.78975, 7...",286,1709,1933,289,442
4,ASPREMONT,06006,"POLYGON ((7.24337 43.80202, 7.24351 43.80193, ...",323,919,4301,472,281


# Ajout des classes d'intervalles et des couleurs correspondantes
_pour uniformisation des légendes sur l'ensemble des cartes et des types de flux_

In [17]:
# choix des classes d'intervalles
ref_bins = [0, 50, 100, 150, 200, 300, 500]
ref_labels = ['< 50', '(50, 100]', '(100, 150]', '(150, 200]', '(200, 300]', '(300, 500]', '> 500']

# création d'un dictionnaire de couleurs pour chaque classe d'intervalle à partir de la cmap 'Oranges'
n_bins = len(ref_bins)
cmap = mpl.colormaps['Oranges']
ref_colors = cmap(np.linspace(0, 1, n_bins))
custom_cmap = mpl.colors.ListedColormap(ref_colors)


color_dict = {}
color_dict[np.nan] = '#808080'
for n in range(n_bins):
    color_dict[ref_labels[n]] = mpl.colors.rgb2hex(custom_cmap(n))

In [18]:
# on ne garde que les bins et les labels nécessaires pour chaque colonne
# cut selon les bins retenus
# assignation des couleurs pour chaque classe
for type in types_dict.values():
    dist_type = 'dist_' + str(type)
    moy_max = round(max(distances_nice[dist_type]))
    bins = [bin for bin in ref_bins if bin < moy_max]
    bins.append(moy_max)
    labels = ref_labels[:len(bins)-1]
    distances_nice[dist_type + '_cat'] = pd.cut(distances_nice[dist_type], bins = bins, labels = ref_labels)
    distances_nice[dist_type + '_col'] = distances_nice[dist_type+'_cat'].map(color_dict)

In [19]:
pav_nice = pav_nice.to_crs(3857)
distances_nice = distances_nice.to_crs(3857)

In [20]:
pav_nice.head()

,COMMUNE,JOUR,ADRESSE,Type de flux,ID_NCA,TYPE,geometry
0,Nice,Lundi,RUE SORGENTINO / RUE MARECHAL VAUBAN,V,48088V,AERIEN,POINT (811729.405 5420817.29)
1,Nice,Lundi,CHEMIN DU MONT GROS / BD BISCHOFFSHEIM (DANS C...,V,49088V,AERIEN,POINT (812407.309 5421812.557)
2,Nice,Lundi,BOULEVARD ARMEE DES ALPES FACE AU N°22,V,50088V,AERIEN,POINT (811821.425 5420373.407)
3,Nice,Lundi,BD ARMEE DES ALPES / RUE LAURENT GIAUME,V,51088V,AERIEN,POINT (811771.263 5420162.143)
4,Nice,Lundi,BOULEVARD PIERRE SOLA ANGLE RUE BEAUTRUCH,V,52088V,AERIEN,POINT (811468.373 5420058)


In [21]:
distances_nice.head()

,NOM,CODE_INSEE,geometry,dist_V,dist_OM,dist_T,dist_P,dist_E,dist_V_cat,dist_V_col,dist_OM_cat,dist_OM_col,dist_T_cat,dist_T_col,dist_P_cat,dist_P_col,dist_E_cat,dist_E_col
0,LE BROC,06025,"POLYGON ((796154.935 5441571.416, 796182.414 5...",319,2020,818,352,713,"(300, 500]",#b63c02,> 500,#7f2704,> 500,#7f2704,"(300, 500]",#b63c02,> 500,#7f2704
1,BONSON,06021,"POLYGON ((800626.805 5446311.617, 800622.099 5...",276,3115,2897,279,356,"(200, 300]",#e95e0d,> 500,#7f2704,> 500,#7f2704,"(200, 300]",#e95e0d,"(300, 500]",#b63c02
2,GILETTE,06066,"POLYGON ((794274.842 5444376.3, 794285.44 5444...",246,2718,1455,246,280,"(200, 300]",#e95e0d,> 500,#7f2704,> 500,#7f2704,"(200, 300]",#e95e0d,"(200, 300]",#e95e0d
3,GATTIERES,06064,"POLYGON ((794155.468 5433006.286, 794187.094 5...",286,1709,1933,289,442,"(200, 300]",#e95e0d,> 500,#7f2704,> 500,#7f2704,"(200, 300]",#e95e0d,"(300, 500]",#b63c02
4,ASPREMONT,06006,"POLYGON ((806328.655 5434855.066, 806344.278 5...",323,919,4301,472,281,"(300, 500]",#b63c02,> 500,#7f2704,> 500,#7f2704,"(300, 500]",#b63c02,"(200, 300]",#e95e0d


# Création de la carte

In [22]:
legend_colors = color_dict
if np.nan in legend_colors.keys():
    legend_colors.pop(np.nan)

In [23]:
# fond de carte
map = leafmap.Map(
    location = [43.95, 7.22],
    tiles = 'cartodbpositron',
    zoomSnap = 0.1,
    zoom_start = 10.3,
    max_zoom=17,
    control_scale=True
    )



# ajout des couches pour chaque type de flux
layers = {}

for n,type in enumerate(types_dict.values()):
    layer_name = 'fg_'+ type 
    label = list(types_dict.keys())[n]
    layers[layer_name] = folium.FeatureGroup(name = label).add_to(map)

    pav_nice.loc[pav_nice['Type de flux']==type].explore(
        m = layers[layer_name],
        marker_kwds=dict(radius=0.5, fill=True),
        tooltip = ['COMMUNE'],
        color = 'Gray'
    )


    distances_nice.explore(
        m = layers[layer_name],
        color = 'dist_'+type+'_col',
        tooltip = ['NOM', 'dist_'+type],
        style_kwds = {
            'color' : 'Black',
            'fillOpacity' : 0.8
            }
    )

    
    map.add_child(layers[layer_name])


# Contrôle de l'affichage des couches
GroupedLayerControl(
    groups={'Type de flux': [layer for layer in layers.values()]},
    collapsed=False,
).add_to(map)


# ajout de la légende
legend_style = {
    "position": "fixed",
    "z-index": "9999",
    "border": "2px solid grey",
    "background-color": "rgba(255, 255, 255, 0.8)",
    "border-radius": "10px",
    "padding": "5px",
    "font-size": "14px",
    "bottom": "20px",
    "right": "5px",
}

map.add_legend(
    title="distance moyenne", legend_dict=color_dict, draggable=False, style=legend_style
)


# ajout du titre
map_title = "Distance moyenne au pav le plus proche par quartier - NICE"
title_html = f'<h1 align="center" style="font-size:24px" >{map_title}</h1>'
map.get_root().html.add_child(folium.Element(title_html))


map.save('../cartes/map_distance_moyenne_Nice.html')